# 애널리스트 보고서

## 환경 설정

`(1) Env 환경변수`

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
print(os.getenv('LANGCHAIN_TRACING_V2'))

true


## PyMuPDFLoader

출처: https://nts.go.kr/comm/nttFileDownload.do?fileKey=15f1850775866b127ee024dbe70f623d

In [3]:
pdf_file = os.path.join('data', '20240524_company_234017000.pdf')
pdf_file

'data\\20240524_company_234017000.pdf'

In [4]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(pdf_file, extract_images=True)
data = loader.load()
len(data)

8

In [5]:
from pprint import pprint
pprint(data[0].page_content)

('  \n'
 ' \n'
 'Korea | 제약 바이오 | 2024.05.24 \n'
 '셀트리온 \n'
 '(068270) \n'
 ' \n'
 '투자의견 \n'
 'BUY(유지) \n'
 '목표주가 \n'
 '250,000 원(유지) \n'
 '현재주가 \n'
 '183,500 원(05/23) \n'
 '시가총액 \n'
 '39,818(십억원) \n'
 '[2024 DDW] A Quick Recap \n'
 ': 짐펜트라, 시간의 문제   \n'
 ' \n'
 '제약/바이오  권해순_ 02)368-6181_ hskwon@eugenefn.com \n'
 ' \n'
 '\uf06e 투자의견 BUY 및 목표주가 25만원 유지, 적극적으로 매수해야 할 때라고 판단함  \n'
 '\uf06e 밸류체인 일원화와 미국 시장에서 유통 채널망을 구축하며 비즈니스모델 혁신(Business model Innovation) '
 '진행 중  \n'
 '\uf06e 5월 18~21일 개최된 2024 DDW(Digestive Disease Week)를 참관하여 IBD(염증성 장질환) '
 '치료제들의 글로벌 개\n'
 '발 동향 및 마케팅 현황을 파악함  \n'
 '\uf06e IBD 치료제 시장 동향  \n'
 '① 염증성 장질환 유병률이 상승하면서 신제품 출시, 제형 변화, 바이오시밀러 출시로 IBD 치료제 시장 격변 중 \n'
 '② 다케다, J&J, 애브비, 그리고 국내 제약사 셀트리온이 시장 주도권을 확보하기 위해 치열한 마케팅 진행 \n'
 '\uf06e 셀트리온: 짐펜트라로 외형 성장과 이익 성장 두 마리 토끼를 잡는다  \n'
 '① 제형 변화로 편의성은 기본, 치료 효과와 안전성까지 모두 확보: 2년 연장 임상에서 우수한 약효 유지 효과 확인 \n'
 '② 2024 DDW에서 짐펜트라에 대한 의료진들의 높은 관심과 우호적인 반응을 확인함, 밀착 마케팅 진행 중  \n'
 '③ TNF-α 치료제 시장에서 점유율 상승에 따른 매출 증가는 시간의 문제 \n'
 ' \n

## Unstructured
https://unstructured.io/

In [17]:
from unstructured.partition.pdf import partition_pdf


# Get elements
pdf_elements = partition_pdf(
    filename=pdf_file,
    extract_images_in_pdf=True,
    infer_table_structure=True,
    languages=["eng", "kor"],
)

len(pdf_elements)

125

In [18]:
pdf_elements[0]

In [19]:
pprint(pdf_elements[0].text)

('    Korea | 제약 바이오 | 2024.05.24      투자의견  BUY(유지)   셀트리온  (068270)   목표주가  '
 '250,000 원(유지)   현재주가  183,500 원(05/23)   [2024 DDW] A Quick Recap  :  짐펜트라,  '
 '시간의  문제       시가총액  39,818(십억원)      제약/바이오  권해순_ 02)368-6181_ '
 'hskwon@eugenefn.com  ')


In [20]:
pprint(pdf_elements[0].metadata.to_dict())

{'coordinates': {'layout_height': 2339,
                 'layout_width': 1654,
                 'points': ((-4.166666666666666, -35.27777777777759),
                            (-4.166666666666666, 735.4166666666666),
                            (1657.2222222222222, 735.4166666666666),
                            (1657.2222222222222, -35.27777777777759)),
                 'system': 'PixelSpace'},
 'file_directory': 'data',
 'filename': '20240524_company_234017000.pdf',
 'filetype': 'application/pdf',
 'image_path': 'c:\\Users\\redwi\\langchain\\langchain_rag\\figures\\figure-1-1.jpg',
 'languages': ['eng', 'kor'],
 'last_modified': '2024-06-23T15:52:36',
 'page_number': 1}


## Multimodal

In [21]:
import base64

def encode_image_to_base64(image_path):
    """
    주어진 이미지 파일 경로를 base64로 인코딩하여 반환합니다.
    
    Args:
    - image_path (str): 이미지 파일 경로
    
    Returns:
    - str: base64로 인코딩된 이미지 데이터
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# PDF 요소에서 이미지 경로를 가져옴
image_path = pdf_elements[0].metadata.to_dict().get('image_path')

# 이미지를 base64로 인코딩
image_data = encode_image_to_base64(image_path)

print(image_data)


...


### GPT-4o

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Describe the image provided and its contents. Include all of the texts in the image. Answer in KOREAN."),
        (
            "user",
            [
                {
                    "type": "image_url",
                    "image_url": {"url": "data:image/jpeg;base64,{image_data}"},
                }
            ],
        ),
    ]
)

chain = prompt | model

response = chain.invoke({"image_data": image_data})
pprint(response.content)


('이 이미지는 유진투자증권의 주식 분석 정보입니다. 주요 내용은 다음과 같습니다:\n'
 '\n'
 '- 제목: 셀트리온 (068270)\n'
 '- 날짜: 2024.05.24\n'
 '- 주제: [2024 DDW] A Quick Recap : 짐펜트라, 시간의 문제\n'
 '- 투자 의견: BUY (유지)\n'
 '- 목표 주가: 250,000 원 (유지)\n'
 '- 현재 주가: 183,500 원 (05/23)\n'
 '- 시가 총액: 39,818 (십억원)\n'
 '- 담당자: 권해순\n'
 '- 연락처: 02)368-6181, hskwon@eugenefn.com')


In [26]:
pprint(response.response_metadata)

{'finish_reason': 'stop',
 'logprobs': None,
 'model_name': 'gpt-4o-2024-05-13',
 'system_fingerprint': 'fp_5e997b69d8',
 'token_usage': {'completion_tokens': 153,
                 'prompt_tokens': 1479,
                 'total_tokens': 1632}}


### GPT-4o-mini

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model2 = ChatOpenAI(model="gpt-4o-mini")


chain2 = prompt | model2

response2 = chain2.invoke({"image_data": image_data})
pprint(response2.content)


('이미지에는 유진투자증권의 정보가 포함되어 있습니다. 주요 내용은 다음과 같습니다:\n'
 '\n'
 '- **회사명:** 셀트리온 (068270)\n'
 '- **투자의견:** BUY (유지)\n'
 '- **목표주가:** 250,000 원 (유지)\n'
 '- **현재주가:** 183,500 원 (2023년 5월 23일 기준)\n'
 '- **시가총액:** 39,818 (십억 원)\n'
 '- **기타 정보:** [2024 DDW] A Quick Recap : 짐펜트라, 시간의 문제\n'
 '\n'
 '하단에는 연락처 정보가 포함되어 있습니다.')


In [27]:
pprint(response2.response_metadata)

{'finish_reason': 'stop',
 'logprobs': None,
 'model_name': 'gpt-4o-mini-2024-07-18',
 'system_fingerprint': 'fp_7dd529cfca',
 'token_usage': {'completion_tokens': 142,
                 'prompt_tokens': 48203,
                 'total_tokens': 48345}}
